In [1]:
import os
import sys
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


In [2]:
import torch
from utils.model import HockeyActionModelResNet18, HockeyActionModelResNet34, R3D18Model
from utils.dataloader import HockeyTestDataset
from torch.utils.data import DataLoader
import pandas as pd
import os
import numpy as np
from torchvision import transforms
from PIL import Image

# --- Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Test-Daten laden ---
test_dataset = HockeyTestDataset(
    csv_file='data/labels_test.csv',
    frames_root='data/test_frames/',
    frames_per_clip=100
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# --- Modellpfade und Klassen ---
model_paths = {
    'ResNet18_LSTM': 'models/best_resnet18_lstm.pth',
    'ResNet34_LSTM': 'models/best_resnet34_lstm.pth',
    'R3D18': 'models/best_r3d18.pth'
}

model_classes = {
    'ResNet18_LSTM': HockeyActionModelResNet18,
    'ResNet34_LSTM': HockeyActionModelResNet34,
    'R3D18': R3D18Model
}

# --- Evaluation ---
for model_name in model_paths:
    print(f"\nPredicting with {model_name}...")

    # Modell initialisieren mit passenden Parametern
    if model_name == 'ResNet18_LSTM':
        model = HockeyActionModelResNet18(embed_size=128, hidden_size=256, num_classes=4)
    elif model_name == 'ResNet34_LSTM':
        model = HockeyActionModelResNet34(num_classes=4)
    elif model_name == 'R3D18':
        model = R3D18Model(num_classes=4)
    else:
        raise ValueError(f"Unbekanntes Modell: {model_name}")

    # Modellgewichte laden
    model.load_state_dict(torch.load(model_paths[model_name], map_location=device), strict=False)
    model = model.to(device)
    model.eval()

    results = []

    with torch.no_grad():
        for videos, clip_names in test_loader:
            videos = videos.to(device)  # [B, T, 3, 224, 224]

            # Spezielle Form für R3D18
            if model_name == 'R3D18':
                videos = videos.permute(0, 2, 1, 3, 4)  # [B, 3, T, 224, 224]

            outputs = model(videos)
            outputs = torch.sigmoid(outputs)
            preds = (outputs > 0.5).cpu().numpy().astype(int)

            for clip_name, pred in zip(clip_names, preds):
                labels = []
                if pred[0] == 1:
                    labels.append('Check')
                if pred[1] == 1:
                    labels.append('Neutral')
                if pred[2] == 1:
                    labels.append('Schuss')
                if pred[3] == 1:
                    labels.append('Tor')

                label_string = ",".join(labels) if labels else "None"
                results.append({
                    'clip_name': clip_name,
                    'predicted_labels': label_string
                })

    # Ergebnisse speichern
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'predictions_{model_name}.csv', index=False)
    print(f"Prediction CSV saved as predictions_{model_name}.csv")



Predicting with ResNet18_LSTM...


c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
C:\Users\hp\AppData\Local\Temp\ipykernel_8408\3566072563.py:50: FutureWarning: You 

Prediction CSV saved as predictions_ResNet18_LSTM.csv

Predicting with ResNet34_LSTM...


c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Prediction CSV saved as predictions_ResNet34_LSTM.csv

Predicting with R3D18...


c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Prediction CSV saved as predictions_R3D18.csv
